In [94]:
import pandas as pd
import numpy as np
from date_continuity_check import pandas_inferred_frequency,frequency,is_continuous

In [11]:
#data folder path definition
data_path="c:\\Users\\seyma\\personal_work\\sanofi\\Data\\"

In [84]:
#first load the excel document
Excel_file=pd.ExcelFile(data_path+"Datainput.xlsx")

#create data frames for each sheet in the excel file
Combined_Yields=pd.read_excel(Excel_file,sheet_name="Combined_Yields")
QC_Data=pd.read_excel(Excel_file,sheet_name="QC_Data")
IoT=pd.read_excel(Excel_file,sheet_name="IoT")
Campaign_Info=pd.read_excel(Excel_file,sheet_name="Campaign_Info")
Batch_Genealogy=pd.read_excel(Excel_file,sheet_name="Batch_Genealogy")

In the Document, they said *"You can use the “activity_coefficient” (column YIELD_NAME) of these output batches as the target in your model"*


So, Let's begin with filtering materials who has YIELD_NAME equal to “activity_coefficient”

In [30]:
Combined_Yields[Combined_Yields.YIELD_NAME=="activity_coefficient"]

,BATCH_ID,MATERIAL_ID,MANUFACTURE_DATE,STAGE,YIELD_NAME,YIELD_VALUE
0,120932,P10446,20200612,s2,activity_coefficient,1.335753
1,120928,P10444,20200924,s2,activity_coefficient,1.275753
2,120935,P10444,20201015,s2,activity_coefficient,1.299432
3,120928,P10446,20200511,s2,activity_coefficient,1.324042
4,119136,P10344,20190329,s2,activity_coefficient,1.332979
...,...,...,...,...,...,...
168,122826,P10449,20221006,s2,activity_coefficient,1.124211
169,122810,P10148,20220831,s2,activity_coefficient,NaN
170,122828,P10449,20221028,s2,activity_coefficient,1.105905
171,122829,P10449,20221117,s2,activity_coefficient,1.134234


We have 173 data points but 65 of them have missing value on target variable.

In [31]:
Combined_Yields[Combined_Yields.YIELD_NAME=="activity_coefficient"].isna().sum()

BATCH_ID             0
MATERIAL_ID          0
MANUFACTURE_DATE     0
STAGE                0
YIELD_NAME           0
YIELD_VALUE         65
dtype: int64

I have decided to dropping those missing values. However, there may some ways to fill them but it is time consuming.

In [34]:
Combined_Yields_full=Combined_Yields[Combined_Yields.YIELD_NAME=="activity_coefficient"].dropna()

In [36]:
Combined_Yields_full.isna().sum()

BATCH_ID            0
MATERIAL_ID         0
MANUFACTURE_DATE    0
STAGE               0
YIELD_NAME          0
YIELD_VALUE         0
dtype: int64

I will join __Campaign_Info table__ with __my Combined_Yields_full__ to use campain id and batch position

In [37]:
Campaign_Info

,BATCH_ID,MATERIAL_ID,CAMPAIGN_ID,BATCH_POSITION
0,119114.0,P10343,12.0,1.0
1,122803.0,P10450,62.0,1.0
2,121831.0,P10449,37.0,1.0
3,121832.0,P10449,37.0,2.0
4,122828.0,P10449,63.0,1.0
...,...,...,...,...
152,122817.0,P10449,48.0,2.0
153,119133.0,P10344,3.0,1.0
154,119134.0,P10344,3.0,2.0
155,119135.0,P10344,3.0,3.0


In [43]:
joined_df = pd.merge(Combined_Yields_full, Campaign_Info, on=['BATCH_ID', 'MATERIAL_ID'])

In [44]:
joined_df.isna().sum()

BATCH_ID            0
MATERIAL_ID         0
MANUFACTURE_DATE    0
STAGE               0
YIELD_NAME          0
YIELD_VALUE         0
CAMPAIGN_ID         7
BATCH_POSITION      7
dtype: int64

In [45]:
joined_df=joined_df.dropna()

In [46]:
joined_df.isna().sum()

BATCH_ID            0
MATERIAL_ID         0
MANUFACTURE_DATE    0
STAGE               0
YIELD_NAME          0
YIELD_VALUE         0
CAMPAIGN_ID         0
BATCH_POSITION      0
dtype: int64

In [47]:
joined_df

,BATCH_ID,MATERIAL_ID,MANUFACTURE_DATE,STAGE,YIELD_NAME,YIELD_VALUE,CAMPAIGN_ID,BATCH_POSITION
1,120928,P10444,20200924,s2,activity_coefficient,1.275753,30.0,1.0
2,120935,P10444,20201015,s2,activity_coefficient,1.299432,30.0,5.0
3,120928,P10446,20200511,s2,activity_coefficient,1.324042,27.0,1.0
4,119136,P10344,20190329,s2,activity_coefficient,1.332979,3.0,4.0
5,119146,P10344,20191105,s2,activity_coefficient,1.329679,6.0,1.0
...,...,...,...,...,...,...,...,...
102,122825,P10449,20220916,s2,activity_coefficient,1.094054,58.0,2.0
103,122827,P10449,20221013,s2,activity_coefficient,1.123533,61.0,2.0
104,122826,P10449,20221006,s2,activity_coefficient,1.124211,61.0,1.0
105,122828,P10449,20221028,s2,activity_coefficient,1.105905,63.0,1.0


So, we have IoT sheet, I need to join this table as well. It will be same join on material and batch ids

In [48]:
joined_df = pd.merge(joined_df, IoT, on=['BATCH_ID', 'MATERIAL_ID'])

In [67]:
column_na_counts = joined_df.isna().sum()

# Iterate over the column_na_counts Series and print each column along with its corresponding NaN count
for column, count in column_na_counts.items():
    print(f"{column}: {count}")

BATCH_ID: 0
MATERIAL_ID: 0
MANUFACTURE_DATE: 0
STAGE: 0
YIELD_NAME: 0
YIELD_VALUE: 0
CAMPAIGN_ID: 0
BATCH_POSITION: 0
campagne finance: 0
1_con_dur: 1
1_con_temp: 1
1_fil_dur: 1
1_final PH: 1
2_alka_hydro_dur: 1
2_fil_dur: 1
2_final_PH: 1
3_1a_etoh rto: 1
3_1a_etoh temp: 1
3_1a_hep concent: 1
3_1a_nacl cont: 1
3_1a_settl_dur: 1
3_2a_initial PH: 1
3_2a_etoh rto: 1
3_2a_hep cont: 1
3_2a_nacl cont: 1
3_2b_settl_dur: 0
3_2a_etoh temp: 1
3_2a_temp of hep solution: 1
4_dry extr_lod: 0
4_fil_dur: 0
4_Total vol: 0
4-2_etoh_rto: 0
4-3 etoh cont_Mother_liquor: 0
4-3_etoh cont: 0
stage5_drying_dur: 1
anti_iia_sur_sec: 0
3_2b_vol_red supernatant_UOL: 0
3_2b_vol precipitate: 0
3_2b_temp of hep solution: 0
3_2b_PH_before feeding: 0
3_2b_PH after feeding: 0
3_2b_hep concent: 0
3_2b_etoh_temp: 0
3_2b_etoh_rto: 0
3_2a_vol red_supernatant_UOL: 1
3_2a_vol precipitate: 1
3_2a_total vol after settl: 1
3_2a_settl_dur: 1
3_2a_nacl cont_: 1
3_2a_final PH: 1
3_1b_vol red supernatant_UOL: 23
3_1b_vol precipitat

In [68]:
joined_df

,BATCH_ID,MATERIAL_ID,MANUFACTURE_DATE,STAGE,YIELD_NAME,YIELD_VALUE,CAMPAIGN_ID,BATCH_POSITION,campagne finance,1_con_dur,1_con_temp,1_fil_dur,1_final PH,2_alka_hydro_dur,2_fil_dur,2_final_PH,3_1a_etoh rto,3_1a_etoh temp,3_1a_hep concent,3_1a_nacl cont,3_1a_settl_dur,3_2a_initial PH,3_2a_etoh rto,3_2a_hep cont,3_2a_nacl cont,3_2b_settl_dur,3_2a_etoh temp,3_2a_temp of hep solution,4_dry extr_lod,4_fil_dur,4_Total vol,4-2_etoh_rto,4-3 etoh cont_Mother_liquor,4-3_etoh cont,stage5_drying_dur,anti_iia_sur_sec,3_2b_vol_red supernatant_UOL,3_2b_vol precipitate,3_2b_temp of hep solution,3_2b_PH_before feeding,3_2b_PH after feeding,3_2b_hep concent,3_2b_etoh_temp,3_2b_etoh_rto,3_2a_vol red_supernatant_UOL,3_2a_vol precipitate,3_2a_total vol after settl,3_2a_settl_dur,3_2a_nacl cont_,3_2a_final PH,3_1b_vol red supernatant_UOL,3_1b_vol precipitate,3_1b_total vol,3_1b_temp of hep solution,3_1b_settl_dur,3_1b_initial PH,3_1b_final PH,3_1b_hep concent,3_1b_etoh_temp,3_1b_etoh_rto,3_1a_vol_red supernatant,3_1a_vol_precipitate,3_1a_total_vol,3_1a_temp of hparin solution,3_2b_nacl cont,3_1_etoh_rto,3_1_hep concent,3_1_nacl cont,3_1_settl_dur,3_1_temp of hparin solution,3_1_total_vol,3_1_vol red supernatant_UOL,3_2_etoh_rto,3_2_etoh_temp,3_2_final PH,3_2_hep cont,3_2_nacl cont,3_2_settl_dur,3_2_temp of hep solution,3_2_vol precipitate,3_2_vol_red supernatant_UOL,stage_3_1b_nacl cont,3_2_vol reactor,3_2a_vol reactor,3_2b_vol reactor,fet_3_1a_final PH,fet_3_1a_initial PH
0,120928,P10444,20200924,s2,activity_coefficient,1.275753,30.0,1.0,30,75.0,77.0,681.0,9.1,375.0,130.0,11.1,0.640562,18.0,19.920000,0.1064,733.0,5.0,0.591535,19.846667,0.0257,730.0,18.0,19.0,0.37,280.0,1058.0,1.000000,0.900,0.9,4101.0,189.847716,0.033333,145.0,19.0,5.0,5.3,19.933333,18.0,0.000000,0.0,145.0,150.0,740.0,0.0257,5.3,0.040000,230.0,236.0,19.0,873.0,5.5,5.3,19.913333,18.0,0.640777,0.066667,210.0,220.0,19.0,0.0252,1.281339,19.916667,0.05320,803.0,19.0,228.0,0.053333,0.295768,18.0,5.3,19.890000,0.02545,735.0,19.0,145.0,0.033333,0.0995,5972.0,2977.0,2995.0,5.3,5.4
1,120935,P10444,20201015,s2,activity_coefficient,1.299432,30.0,5.0,30,87.0,78.0,650.0,9.1,371.0,163.0,11.1,0.640741,18.0,19.800000,0.0982,718.0,5.0,0.590726,19.840000,0.0240,723.0,18.0,19.0,0.35,406.0,1123.0,1.000000,0.900,0.9,3948.0,182.839632,0.066667,140.0,19.0,5.0,5.3,19.820000,18.0,0.589976,0.0,140.0,150.0,723.0,0.0240,5.3,0.066667,200.0,210.0,19.0,722.0,8.9,5.3,19.866667,18.0,0.640940,0.066667,190.0,200.0,19.0,0.0256,1.281680,19.833333,0.04910,720.0,19.0,205.0,0.066667,0.590351,18.0,5.3,19.830000,0.02480,723.0,19.0,140.0,0.066667,0.0999,5964.0,2976.0,2988.0,5.3,5.4
2,120932,P10444,20201008,s2,activity_coefficient,1.408540,30.0,3.0,30,154.0,78.0,621.0,9.1,371.0,142.0,11.1,0.639906,18.0,19.846667,0.0982,728.0,5.0,0.590191,19.846667,0.0247,723.0,18.0,19.0,0.35,364.0,1106.0,1.000000,0.900,0.9,3946.0,180.428135,0.066667,140.0,19.0,5.0,5.3,19.946667,18.0,0.590575,0.0,140.0,150.0,726.0,0.0247,5.3,0.033333,200.0,205.0,20.0,723.0,5.4,5.3,19.853333,18.0,0.640698,0.020000,208.0,211.0,20.0,0.0253,1.280604,19.850000,0.04910,725.5,20.0,208.0,0.026667,0.590383,18.0,5.3,19.896667,0.02500,724.5,19.0,140.0,0.066667,0.1020,5974.0,2977.0,2997.0,5.3,5.5
3,120930,P10444,20201001,s2,activity_coefficient,1.395169,30.0,2.0,30,75.0,80.0,615.0,9.1,371.0,126.0,11.1,0.640457,18.0,19.840000,0.1009,720.0,4.9,0.590726,19.840000,0.0240,845.0,18.0,19.0,0.37,413.0,1099.0,1.000000,0.905,0.9,3990.0,186.734694,0.033333,140.0,19.0,4.9,5.3,19.933333,18.0,0.590970,0.0,140.0,146.0,762.0,0.0240,5.3,0.006667,200.0,201.0,19.0,722.0,5.4,5.3,19.840000,18.0,0.640793,0.000000,210.0,210.0,19.0,0.0269,1.281250,19.840000,0.05045,721.0,19.0,205.5,0.003333,0.590848,18.0,5.3,19.886667,0.02545,803.5,19.0,140.0,0.036667,0.1005,5968.0,2976.0,2992.0,5.3,5.5
4,120936,P10449,20201103,s2,activity_coefficient,1.096267,31.0,2.0,31,98.0,78.0,498.0,9.1,371.0,141.0,11.1,0.640620,18.0,19.793333,0.1012,4070.0,5.0,0.640646,19.813333,0.0256,722.0,18.0,19.0,0.34,320.0,1270.0,1.

One more join left. It is QC_Data

In [72]:
QC_Data

,BATCH_ID,MATERIAL_ID,MANUFACTURE_DATE,NEW_QC_TEST,VALUE
0,122875,P10051,20220704,pere_pour_azo,1.8
1,115705,P10005,20170721,cend_sul(fourni),NaN
2,115703,P10144,20171031,lla_fourni,NaN
3,120932,P10010,20200318,PH,6.9
4,122882,P10051,20220829,xa_tel_fourni,NaN
...,...,...,...,...,...
428973,122750,P10147,20201214,sul_ntu(fourni),NaN
428974,116258,P10044,20170109,cend_c1_creuset2,NaN
428975,118367,P10044,20180730,azot(fourni),NaN
428976,117224,P90111,20181127,than_fourni,NaN


In [91]:
#there were some zeros on the "MANUFACTURE_DATE" column. So, replaced them with NaN
QC_Data['MANUFACTURE_DATE'] = QC_Data['MANUFACTURE_DATE'].replace(0, np.nan)

#Transforming type from string to DateTime
QC_Data['MANUFACTURE_DATE'] = pd.to_datetime(QC_Data['MANUFACTURE_DATE'], format='%Y%m%d')

#for better understanding, Sorting values in ASC order
QC_Data = QC_Data.sort_values('MANUFACTURE_DATE')

In [93]:
QC_Data.isna().sum()

BATCH_ID                 0
MATERIAL_ID              0
MANUFACTURE_DATE      4070
NEW_QC_TEST              0
VALUE               372968
dtype: int64

Most of the test result is empty. So I want to see which test metrics have a value

In [100]:
QC_Data[QC_Data["VALUE"].isna()==False]

,BATCH_ID,MATERIAL_ID,MANUFACTURE_DATE,NEW_QC_TEST,VALUE
0,122875,P10051,2022-07-04,pere_pour_azo,1.800000
3,120932,P10010,2020-03-18,PH,6.900000
17,118967,P10003,2018-03-27,anticoagulante,107.000000
20,117273,P10144,2018-04-30,xa_dry,188.574423
22,119638,P10004,2019-05-13,PH,6.800000
...,...,...,...,...,...
428939,118329,P10044,2018-03-01,lod,0.500000
428948,118317,P10044,2018-02-08,xa_dry,200.607903
428950,122020,P10011,2022-01-10,coag_wet,175.000000
428962,115916,P10147,2017-04-04,azot_sec,2.300000


In [104]:
print("List of QC Tests that have value:")
print(QC_Data[QC_Data["VALUE"].isna()==False]["NEW_QC_TEST"].unique())
print("----------------------------------------------------------------")
print("Length of list:")
print(len(QC_Data[QC_Data["VALUE"].isna()==False]["NEW_QC_TEST"].unique()))

List of QC Tests that have value:
['pere_pour_azo' 'PH' 'anticoagulante ' 'xa_dry' 'ratio_xa_lla(sec)' 'lod'
 'dessic_moyenne' 'xa_tel' 'teneur_eau' 'coag_wet' 'masse' 'moyenne'
 'es2_perte' 'pertem2_cre2' 'ethan' 'perte_cre1' 'azot_sec' 'ratio_80_24'
 'adn' 'absorbance' 'lla_tel' 'lla_sec' 'teneur_chon'
 'es2_azo_sur_sec_pct' 'pere_pour_lla' 'proteiniq' 'coag_dry'
 'cend_c1_creuset2' 'cend_sul' 'sul_ntu' 'rende_la' 'dessic_creuset'
 'cend_c1_creuset1' 'ratio_xa_lla(fourni)' 'lines_sec'
 'es1_azo_sur_sec_pct' 'pertem2_cre1' 'otdlq_260' 'sod_sec'
 'cend_m_creuset2' 'es1_azo_sur_tel' 'es2_azo_sur_tel' 'perte_cre2'
 'pertem1cru2' 'perem1cru1' 'cend_c2_creuset2' 'thanol'
 'cend_sul_creuset2' 'perte' 'ratio_xa_llafourni' 'cend_sul_creuset1'
 'pertem1' 'otidiq' 'cend_sul_moy' 'cend_m_creuset1' 'lla_fourni'
 'xa_sec_fourni' 'otidiq_hplc' 'cend_c2_creuset1' 'cal_sec' 'robie_to'
 'rende_uimg' 'pertem2' 'lla_tel_fourni' 'cend_sul(fourni)'
 'xa_tel_fourni' 'sul_ntu(fourni)' 'inque_lowry' 'azot(fo

In [90]:
print(QC_Data["MANUFACTURE_DATE"].max())
print(QC_Data["MANUFACTURE_DATE"].min())

2023-01-16 00:00:00
2014-05-02 00:00:00


In [77]:
unique_combinations = QC_Data[['BATCH_ID', 'MATERIAL_ID','NEW_QC_TEST','MANUFACTURE_DATE']].nunique()
print("# of unique: ")
print(unique_combinations)

# of unique: 
BATCH_ID            5181
MATERIAL_ID           40
NEW_QC_TEST           74
MANUFACTURE_DATE    1224
dtype: int64


In [80]:
#this is for understanding how many different pairs we have
unique_pairs_count = QC_Data[['BATCH_ID', 'MATERIAL_ID']].drop_duplicates().shape[0]
print(unique_pairs_count)


5797


In [ ]:
#joined_df = pd.merge(joined_df, QC_Data, on=['BATCH_ID', 'MATERIAL_ID'])

Batch Genealogy

In [105]:
Batch_Genealogy

,BATCH_ID,MATERIAL_ID,LEVEL,TREE_ID,PROCESS_ORDER_NUMBER,MANUFACTURE_DATE,QUANTITY,UNIT_OF_MEASURE,MOVEMENT_TYPE,NEW_MATERIAL_DESCRIPTION
0,122812,P10449,2,02d31e7f93f5de3cb5b7fcc8ab00426f4f57f7fa33dad5...,R104490026,20220203,232.100,KG,101,hep_so
1,122813,P10449,2,02d31e7f93f5de3cb5b7fcc8ab00426f4f57f7fa33dad5...,R104490027,20220211,253.550,KG,101,hep_so
2,122825,P10449,2,0347d96c7e652f3ac24b2bf917652763581f0dc1b62594...,R104490036,20220916,265.100,KG,101,hep_so
3,121843,P10449,2,035e3927189e6c90047992537b968bde33f46a3b1acb9d...,R104490023,20211105,223.900,KG,101,hep_so
4,121823,P10450,2,035e3927189e6c90047992537b968bde33f46a3b1acb9d...,R104500007,20211021,227.200,KG,101,hep_so
...,...,...,...,...,...,...,...,...,...,...
4192,121907,P10010,2,4f6560e1695aacd541ba25c5ee8bf7ba7b013176c04354...,R104500005,20210106,25.035,KG,261,hep_bru_dc
4193,121908,P10010,2,243b6ba66876069e071084a9ba262c682c42b37e615588...,R104500005,20210113,26.041,KG,261,hep_bru_dc
4194,121908,P10010,2,4f6560e1695aacd541ba25c5ee8bf7ba7b013176c04354...,R104500005,20210113,26.041,KG,261,hep_bru_dc
4195,121909,P10010,2,243b6ba66876069e071084a9ba262c682c42b37e615588...,R104500005,20210114,5.916,KG,261,hep_bru_dc


In [106]:
Batch_Genealogy["MOVEMENT_TYPE"].unique()

array([101, 531, 261], dtype=int64)

In [108]:
Batch_Genealogy["LEVEL"].unique()

array([2], dtype=int64)

We have 3 different types of movement. 531 can be disregarded according to document. Also, Level column is not necessary because it stores only one variable for each row.